# Example 1:

## Imports

In [1]:
from mm_neural_adjoint import NANetwork
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

## Read example data

In [3]:
X = pd.read_csv('data/data_x_tiny.csv', header=None, delimiter=' ')
y = pd.read_csv('data/data_y_tiny.csv', header=None, delimiter=' ')

X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

## Convert to Dataloaders

In [4]:
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into train, validation, and test
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size]
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

## Train

This code cell will train the forward pass of the model.

Model is saved during training.

Geometries -> Spectra

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NANetwork(8, 300, device=device)

EPOCHS = 100

with tqdm(total=EPOCHS, desc='Training Progress') as pbar:
    model.train(EPOCHS, train_loader, val_loader, progress_bar=pbar, save=True)


model.evaluate_geometry(test_loader)

## Load Model

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NANetwork(8, 300, device=device)


model.load('checkpoints/best_model.pt')
print(model.geometry_lower_bound)

Successfully loaded model from checkpoints/best_model.pt
tensor([-1., -1., -1., -1., -1., -1., -1., -1.])


## Normal Prediction

Geometry -> Spectra

In [8]:
spectra = model.predict_spectra(X_tensor[0].unsqueeze(0))
print(spectra.shape)

(1, 300)


## NA Method

Spectra -> Geometry

In [10]:
Xpred_top, Ypred_top, MSE_top = model.predict_geometry(y_tensor[0])
print(Xpred_top.shape, Ypred_top.shape, MSE_top.shape)


(1, 8) (1, 300) (1,)
